<h2> This notebook uses CNN from tf to implement digit recognisation of mnist data sets.


In [1]:
import tensorflow as tf


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_DATA/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
#helper function to initialise weights
def initi_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist);

In [5]:
# helper function to initialise bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [6]:
# helper function to create wrapper around tensorflow conv2d function that creates 2 d convolution
# for given input tensor , filter(or kernel) tensor with srides and padding we want

def conv2d(X,W):
    # X is input tensor , in our case it will be of dimensions 
       #                 [batch_size(number of images in the  batch),height(pixels), width(pixels),channels]
    
    # W is filter --> [filter height,filter width, input channel, outpul channel] 
    # we will use stride of 1 in each direction of X , we have four directions!
    return tf.nn.conv2d(X,W,strides=[1,1,1,1],padding='SAME')


In [7]:
# helper function to create pooling . we will create a wrapper around the tf provided poolng function
# this helper function will do 2 by 2 pooling along the pixels
def max_pool_2by2(X):
    # X-->[batch size, pixels height , pixels width, no of channels(3 for GRB , 1 for grey scale)]
    # ksize = max pool size , since we will do max pool only along individual images'sheight and width 
      #           it will be [1,2,2,1]
    # we will use strides of 2 along images's width and height , so it will be [1,2,2,1] again
    # for padding we will use SAME algorithm
    
    return tf.nn.max_pool(X,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [13]:
# convolution layer
# it will apply weights (W) and bias(b) and the relu activation on it
# we will pasee shape as [width_filter, height_filter,input_channel,output_channel(or number of features will be detected in this convolution layer)]
def convolution(input_x,shape):
    W = initi_weights(shape);
    b = init_bias([shape[3]])
    conv_out = conv2d(input_x,W)+b
    return tf.nn.relu(conv_out)

In [14]:
# we will write our fully connected normal layer now
def normal_fully_connected(input_layer,size):
    input_size= int (input_layer.get_shape()[1]) # we can use 1 or 2 (height or width size, both are same)
    W = initi_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b;



In [15]:
# Placesholders for input images and outpts digits (0 to 9)

X= tf.placeholder(tf.float32,shape=[None,784])
y=tf.placeholder(tf.float32,shape=[None,10])

lets implement our CNN various layers now

<h4> We will implement following arhitecture in the mnist data set in this excercise 

input image --> 5 by 5 filter, 32 feature detection(so 32 filters) --->max pool of 2 by 2
--->5 by 5,64 filters-->max poll 2 by 2 --flatten
</h4>

In [16]:
# 1st layer is our input images
# our input images are flattened , so we will reshape in proper dimensions
# -1 says unknown(we dont know number of images)
# 1 is for gray scale channel
x_images = tf.reshape(X,[-1,28,28,1])


<h3> First convolution and max pooling on it

In [18]:
# now we will do first convolution and max pooling

# in this first convolution layer shape will be [5,5,1,32]
conv_1 = convolution(x_images,shape=[5,5,1,32])
conv1_max_pool = max_pool_2by2(conv_1)


<h3> Second convolution and max pooling on it

In [19]:
# here input channel will be 32 (output of pevious layer) and we will extract 64 features from it
conv_2 = convolution(conv1_max_pool,shape=[5,5,32,64])
conv_2_max_pool = max_pool_2by2(conv_2)


<h3> Flattening out our 4 dimenional data to 1 D for fully connected

In [23]:
# we started with 28 by 28 , then we did two time max pooling of 2 by 2
# so our image size is reduced t 7 by 7 finally(in height and  width pixels)

conv_2_flatten = tf.reshape(conv_2_max_pool,[-1,7*7*64])

# we will use 1024 neurons in this fully connencted layer
fully_connected_layer_one = tf.nn.relu(normal_fully_connected(conv_2_flatten,1024))

In [24]:
# we will apply droput
# ceate a placeholder for dropout and apply it
hold_prob = tf.placeholder(tf.float32)
fully_connected_layer_one_droput = tf.nn.dropout(fully_connected_layer_one,keep_prob=hold_prob)

In [25]:
# finnaly our predictions 
y_pred = normal_fully_connected(fully_connected_layer_one_droput,10)

<h3> Loss function and Optimizer to miminize the loss function

In [27]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [28]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)

In [29]:
init = tf.global_variables_initializer()

<h3> Lets run our session and see the acuracy

In [32]:
steps =5000 

with tf.Session() as sess:
    sess.run(init)
    
    for i in  range(steps):
        # get data from train data set in chunks of 50
        batch_x , batch_y = mnist.train.next_batch(50)
        
        # during training we will use dropuout probability as 50 percent
        sess.run(train,feed_dict={X:batch_x,y:batch_y,hold_prob:0.5})
        
        # lets see our accuracy results every 100 steps
        if i%100==0:
            print("For Step {} ,".format(i))
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
            # convert True, False to 1 , 0 and get percentage  of 1 (matches results)
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            # in test we will not do droput ,we will keep all neurons
            acc_result=sess.run(acc,feed_dict={X:mnist.test.images,y:mnist.test.labels,hold_prob:1.0})
            print(" Accuracy is {}",format(acc_result))
            print('\n')
            
            

For Step 0 ,
 Accuracy is {} 0.08169999718666077


For Step 100 ,
 Accuracy is {} 0.8543000221252441


For Step 200 ,
 Accuracy is {} 0.9035999774932861


For Step 300 ,
 Accuracy is {} 0.9240999817848206


For Step 400 ,
 Accuracy is {} 0.9327999949455261


For Step 500 ,
 Accuracy is {} 0.9416000247001648


For Step 600 ,
 Accuracy is {} 0.9460999965667725


For Step 700 ,
 Accuracy is {} 0.953000009059906


For Step 800 ,
 Accuracy is {} 0.9573000073432922


For Step 900 ,
 Accuracy is {} 0.9613999724388123


For Step 1000 ,
 Accuracy is {} 0.9599999785423279


For Step 1100 ,
 Accuracy is {} 0.9646999835968018


For Step 1200 ,
 Accuracy is {} 0.9648000001907349


For Step 1300 ,
 Accuracy is {} 0.9679999947547913


For Step 1400 ,
 Accuracy is {} 0.968999981880188


For Step 1500 ,
 Accuracy is {} 0.9700000286102295


For Step 1600 ,
 Accuracy is {} 0.9707000255584717


For Step 1700 ,
 Accuracy is {} 0.9725000262260437


For Step 1800 ,
 Accuracy is {} 0.9722999930381775


For St